<div style="border:solid Black 3px; padding: 40px">
    
# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
    
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 
    
План:
- Загрузка данных;
- Предобработка данных (обработка пропусков, дубликатов, поиск аномальных значений, приведение типов данных, категоризация данных);
- Исследование данных (поиск факторов, от которых может зависеть возврат кредита в срок);
- Вывод.

## Импорт библиотек и знакомство с данными

In [64]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [65]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Имеются данные о 21525 заёмщиках, в двух столбцах есть пропуски

## Предобработка данных

### Удаление пропусков

In [67]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. В `days_employed` и `total_income`. `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом

In [68]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`.  Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`

In [69]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости посмотрим медианное значение трудового стажа `days_employed` в днях

In [70]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Оставим их как есть. Тем более этот столбец не понадобится для исследования.

Получим перечень уникальных значений столбца `children`

In [71]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

 В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`

In [72]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Убедимся, что артефакты удалены

In [73]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`

In [74]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [75]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`

In [76]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.

In [77]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их

In [78]:
data.duplicated().sum()

71

In [79]:
data = data.drop_duplicates()

### Категоризация данных

 Разделим заёмщиков по категориям, основываясь на `total_income`, согласно следующим критериям:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.



In [80]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [81]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [82]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [83]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [84]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Нужно сгруппировать данные по количеству детей, найти количество людей и количество должников в каждой группе. 
Затем посчитать процентное соотношение должники/вся группа и отсортировать.

In [85]:
data_grouped_children = data.groupby('children').agg({'debt': ['count', 'sum']})
data_grouped_children['debt_percentage'] = data_grouped_children['debt']['sum'] / data_grouped_children['debt']['count']
data_grouped_children.sort_values(by = 'debt_percentage') 

debt       debt_percentage
          count   sum                
children                             
5             9     0        0.000000
0         14091  1063        0.075438
3           330    27        0.081818
1          4808   444        0.092346
2          2052   194        0.094542
4            41     4        0.097561

Вывод: Согласно полученным результатам, самые добросовестные заёмщики имеют 5 детей, однако таких всего 9 человек из 21525, поэтому однозначно утверждать нельзя. Похожая ситуация с клиентами с 3 и 4 детьми. Но в целом, прослеживается такая тенденция: чем больше детей, тем ниже вероятность возврата кредита в срок.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [86]:
data_family_pivot = data.pivot_table(index=['family_status', 'family_status_id'], values='debt', aggfunc={'debt': ['count', 'sum']})
data_family_pivot

,,count,sum
family_status,family_status_id,,
Не женат / не замужем,4,2796,273
в разводе,3,1189,84
вдовец / вдова,2,951,63
гражданский брак,1,4134,385
женат / замужем,0,12261,927


Сделала сводную таблицу 

In [87]:
data_family_pivot['debt_percentage'] = data_family_pivot['sum']/data_family_pivot['count']
data_family_pivot.sort_values('debt_percentage')

,,count,sum,debt_percentage
family_status,family_status_id,,,
вдовец / вдова,2,951,63,0.066246
в разводе,3,1189,84,0.070648
женат / замужем,0,12261,927,0.075606
гражданский брак,1,4134,385,0.093130
Не женат / не замужем,4,2796,273,0.097639


Оценила долю клиентов с задолженностями в каждой группе

Вывод: В данном случае количество человек в группах отличается не так сильно, как в предыдущем задании, однако всё равно группа 0 примерно в 12 раз больше группы 2. Если не учитывать 2 группу, то можно сказать, что у клиентов, которые не женаты/не замужем, самая большая доля задолженников, а у клиентов, которые в разводе, - самая маленькая.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [88]:
data_income_pivot = data.pivot_table(index=['total_income_category'], values='debt', aggfunc={'debt': ['count', 'sum']})
data_income_pivot['debt_percentage'] = data_income_pivot['sum']/data_income_pivot['count']
data_income_pivot

,count,sum,debt_percentage
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


Из таблицы видно, что количество клиентов в категориях сильно различаются. Добавим столбец, который расшифрует эти категории.

In [89]:
data_income_pivot['total_income'] = ['>1000001', '200001–1000000', '50001–200000', '30001–50000', '0–30000']
data_income_pivot.sort_values('debt_percentage')


,count,sum,debt_percentage,total_income
total_income_category,,,,
D,349,21,0.060172,30001–50000
B,5014,354,0.070602,200001–1000000
A,25,2,0.080000,>1000001
C,15921,1353,0.084982,50001–200000
E,22,2,0.090909,0–30000


Вывод: Видим, что суммарно на долю клиентов с зп до 50000 и от 1000000 приходится около 2%, примерно 3/4 всех заёмщиков составляют люди с зп 50001–200000, остальные с зп 200001–1000000. Так как большая часть клиентов приходятся на категорию C и B, по ним можно сказать, что с ростом уровня дохода, уменьшается доля должников. 

#### Как разные цели кредита влияют на его возврат в срок?

In [90]:
data_purpose_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc={'debt': ['count', 'sum']})
data_purpose_pivot['debt_percentage'] = data_purpose_pivot['sum']/data_purpose_pivot['count']
data_purpose_pivot.sort_values('debt_percentage')

,count,sum,debt_percentage
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


Вывод: В данном случае количество клиентов в категориях отличаются меньше, чем в предыдущих исследованиях, поэтому можно сказать, что цели на кредит влияют на его возврат в срок больше, чем уровень дохода, семейное положение и количество детей. Самая большая доля должников среди тех, кто берет кредит на операции с автомобилем, самая малая - среди тех, кто берет кредит на операции с недвижимостью. 

#### Какие возможные причины появления пропусков в исходных данных?

Ответ: Пропуски наблюдаются только в столбцах со стажем и с месячной зарплатой, причём их одинаковое количество (2174). Проверим, находятся ли эти пропуски в одной строке.

In [91]:
import pandas as pd

try:
    data_na = pd.read_csv('/datasets/data.csv')
except:
    data_na = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
   

In [92]:
 
    data_na = data_na.loc[(data_na['days_employed'].isna()) & (data_na['total_income'].isna())]
    data_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Получили таблицу, в которой в столбцах 'days_employed' и 'total_income' находятся NaN. Количество строк в этой таблице также 2174. Значит, можно сделать вывод о том, что эти пропуски между собой связаны. То есть, если появился в одном столбце, значит появится и в другом. Такое не может быть связано с нежеланием людей указывать какую-то информацию (тогда бы пропуски были и в других столбцах и их было бы разное количество). Это может быть объяснятся тем, что 2174 человека получили другой бланк для анкетирования, но вероятнее всего произошёл технический сбой при выгрузке данных.

#### Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных

Найдём максимальную и минимальную зарплату для каждого типа занятости

In [93]:
incomes = []
columns = ['Тип занятости', 'Минимальная зп', 'Максимальная зп']
for t in data['income_type'].unique():
    income = []
    income.append(t)
    income.append(data.loc[data['income_type'] == t, 'total_income'].min())
    income.append(data.loc[data['income_type'] == t, 'total_income'].max())
    incomes.append(income)
income_type = pd.DataFrame(data = incomes, columns = columns)    
income_type

,Тип занятости,Минимальная зп,Максимальная зп
0,сотрудник,21367,1726276
1,пенсионер,20667,735103
2,компаньон,28702,2265604
3,госслужащий,29200,910451
4,безработный,59956,202722
5,предприниматель,499163,499163
6,студент,98201,98201
7,в декрете,53829,53829



Видим, что даже в пределах одного типа занятости доходы могут отличаться на порядки, в таком случае среднее арифметическое даёт некорректное значение.

Повторим то же самое для нахождения минимального и максимального стажа внутри одного типа занятости:

In [94]:
days = []
columns = ['Тип занятости', 'Минимальный стаж', 'Максимальный стаж']
for t in data['income_type'].unique():
    days_employed = []
    days_employed.append(t)
    days_employed.append(data.loc[data['income_type'] == t, 'days_employed'].min())
    days_employed.append(data.loc[data['income_type'] == t, 'days_employed'].max())
    days.append(days_employed)
days_employed_type = pd.DataFrame(data = days, columns = columns)    
days_employed_type

,Тип занятости,Минимальный стаж,Максимальный стаж
0,сотрудник,24.141633,18388.949901
1,пенсионер,328728.720605,401755.400475
2,компаньон,30.195337,17615.563266
3,госслужащий,39.954170,15193.032201
4,безработный,337524.466835,395302.838654
5,предприниматель,520.848083,520.848083
6,студент,578.751554,578.751554
7,в декрете,3296.759962,3296.759962


Видим похожую ситуацию: есть сотрудники со стажем 24 дня и сотрудники со стажем 18388 дней. Поэтому здесь также используем медианное значение.

### Общий вывод.

В результате исследования выяснили:
1) Из-за технических неполадок возникли пропуски в столбцах 'days_employed' и 'total_income' 
2) Среди клиентов с большим количеством детей больше задолженников
3) Среди клиентов в разводе задолженников меньше всего, а среди неженатых - больше всего
4) С ростом уровня дохода уменьшается количество должников
5) Задолженников меньше всего среди клиентов, взявших кредит на операции с недвижимостью, и больше всего среди клиентов, взявших кредит на операции с автомобилем